In [1]:
import numpy as np
import pandas as pd
import sys
import seaborn as sns
import os.path
import time, itertools, re
import importlib
from collections import Counter
import scipy as sc
import miceforest as mf
from sklearn.metrics import make_scorer, balanced_accuracy_score, confusion_matrix, root_mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.pipeline import make_pipeline, Pipeline
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
print(os.getcwd())
fileloc_data='/'.join(os.getcwd().split('/')[0:5])+ '/data/annonymizedDatasets/'
savetag='pred_lavSQ_MHC'
print(fileloc_data)
code_path='/'.join(os.getcwd().split('/')[0:4])+'/sklvq/'
sys.path.append(code_path)
from sklvq import GMLVQ, LGMLVQ
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from EDdataset_GGZ import colsTypeCast    
df_all_combo=pd.read_csv(fileloc_data+'maskedDAIsy_MainDect_ED_SQ48_MHC_Honos_Lav.tsv', sep='\t', decimal=',')
df_adapted_combo, colsExtracted, subscales=colsTypeCast(df_all_combo)
adapted_combo_cols=np.setdiff1d(colsExtracted,
        ['ED_Codes','EDtype', 'SQ48-Score', 'MHCSF-Score', 'Lav-Score']+list(subscales['Honos'])+list(subscales['EDEQ']))
print(adapted_combo_cols)

/home/sreejita/anaconda3/envs/py_env1/lib/python3.10/site-packages/dask/dataframe/__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


/home/sreejita/ProjectsPy/GGZ/code_scripts/ED-profiling
/home/sreejita/ProjectsPy/GGZ/data/annonymizedDatasets/
['DT-BMI' 'DT-Disorder_Duration_Yrs' 'DT-IND_BDL_CMD' 'DT-IND_OCD_CMD'
 'DT-IND_depression_CMD' 'DT-IND_others' 'DT-IND_prev_spec_int_wo_eff'
 'DT-num_prev_routes' 'Lav-Dissatisfaction_body'
 'Lav-Negative_appraisal_body' 'Lav-Unfamiliarity_with_body'
 'MHCSF-Emotional_Well-being' 'MHCSF-Psychological_Well-being'
 'MHCSF-Social_Well-being' 'Main-Age' 'Main-Biosex' 'Main-Education'
 'SQ48-Agoraphobia' 'SQ48-Anxiety' 'SQ48-Cognitive_Complaints'
 'SQ48-Depression' 'SQ48-Hostility' 'SQ48-Social_phobia'
 'SQ48-Somatic_Complaints' 'SQ48-Vitality' 'SQ48-Work_related_complaints']


In [2]:
%load_ext autoreload

# Classification

In [3]:

df_train_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Train']
YTrain=df_adapted_combo['EDtype'].loc[df_adapted_combo['Split']=='Train']#.to_numpy()
df_test_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Test']
YTest=df_adapted_combo['EDtype'].loc[df_adapted_combo['Split']=='Test']#.to_numpy()

nan_mean=np.nanmean(df_train_adapted.to_numpy(), axis=0)
nan_std=np.nanstd(df_train_adapted.to_numpy(), axis=0)
z_train_explore_nan=((df_train_adapted.to_numpy()-nan_mean)/nan_std)
z_test_df=pd.DataFrame(data=((df_test_adapted.to_numpy()-nan_mean)/nan_std), columns=adapted_combo_cols)
z_train_df=pd.DataFrame(data=z_train_explore_nan, columns=adapted_combo_cols)


kernel_mean_match = mf.ImputationKernel(data=z_train_df,num_datasets=1,mean_match_candidates=5)
kernel_mean_match.mice(10)
z_train_explore=pd.DataFrame(data=kernel_mean_match.complete_data(), columns=adapted_combo_cols)
temp_test=kernel_mean_match.impute_new_data(z_test_df)
z_test_explore=pd.DataFrame(data=temp_test.complete_data(), columns=adapted_combo_cols)
print(z_train_explore.shape, z_test_explore.shape)

(97, 26) (35, 26)


In [4]:
%autoreload 2
from HypOpt import GridSearchClassifiers, GridSearch_LVQ
sampling_strategy='not majority'
param_grid={'RF_Max_Features': [5,7,10, 20,25], 'RF_n_Trees':[100, 300, 500], 'RF_min_leaf':5,
           'scorer_name':'Lowest_CWA'}
GridSearchClassifiers(z_train_explore, YTrain,sampling_strategy, param_grid)

RF: 
     n_Trees  Max_Features  Lowest_CWA
7       300            10    0.578788
4       300             7    0.567424
10      300            20    0.563131
6       100            10    0.562626
KNN: 
    K Dist Metric  Lowest_CWA
3  3      cosine    0.472475
1  5   minkowski    0.422475
4  5      cosine    0.421970
0  3   minkowski    0.421465
LDA: 
   Solver  Lowest_CWA
2  eigen    0.473485
0    svd    0.472475
1   lsqr    0.472475
3  eigen         NaN
Linear SVM: 
        C  Lowest_CWA
4   10.0    0.603788
5  100.0    0.603788
3    1.0    0.590657
2    0.1    0.519192
RBF SVM: 
         C  Gamma  Lowest_CWA
32  100.0  0.010    0.637626
26   10.0  0.010    0.600000
31  100.0  0.001    0.556566
20    1.0  0.010    0.429293


In [6]:
from imblearn.over_sampling import SMOTE
pipeRF=Pipeline(steps=[('RF', RandomForestClassifier(criterion="gini", min_samples_leaf=5, 
                        class_weight='balanced_subsample',n_estimators=500, max_features=10))])
labs=np.unique(YTrain)
sm=SMOTE(sampling_strategy=sampling_strategy)
X,Y=sm.fit_resample(z_train_explore, YTrain)
pipeRF.fit(X,Y)
print(confusion_matrix(YTrain, pipeRF.predict(z_train_explore), normalize='true', labels=labs))
print(confusion_matrix(YTest, pipeRF.predict(z_test_explore), normalize='true', labels=labs))

[[0.97142857 0.02857143 0.         0.         0.        ]
 [0.05       0.9        0.         0.05       0.        ]
 [0.07692308 0.07692308 0.84615385 0.         0.        ]
 [0.05882353 0.17647059 0.         0.76470588 0.        ]
 [0.08333333 0.         0.         0.         0.91666667]]
[[0.5        0.0625     0.         0.125      0.3125    ]
 [0.125      0.625      0.125      0.125      0.        ]
 [0.         0.33333333 0.         0.         0.66666667]
 [0.         0.         0.2        0.6        0.2       ]
 [0.         0.33333333 0.         0.         0.66666667]]


In [7]:
param_grid_lvq={'prot_choice': [np.array([3,3,3,1,1]),np.array([3,2,2,1,1]),
            np.array([2,2,2,1,1]),np.array([1,1,1,1,1]), np.array([1,1,2,1,1])],
               'n_comp':[5,10, 20,26], 'scorer_name':'Lowest_CWA'}
GridSearch_LVQ(z_train_explore, YTrain,sampling_strategy, param_grid_lvq)

GMLVQ: 
         Prot per cls Solver  Num Component Activation type  Lowest_CWA
98   [3, 2, 2, 1, 1]  lbfgs              5           swish    0.547980
106  [3, 2, 2, 1, 1]  lbfgs             20           swish    0.542929
82   [3, 3, 3, 1, 1]  lbfgs              5           swish    0.531061
99   [3, 2, 2, 1, 1]   adam              5           swish    0.530808


In [17]:
pipeGMLVQ=Pipeline(steps=[('GMLVQ', GMLVQ(distance_type='adaptive-squared-euclidean', 
            prototype_n_per_class=np.array([3, 2, 2, 1, 1]), relevance_regularization=0.001,
            activation_type='swish',relevance_n_components=20, solver_type='lbfgs'))])
pipeGMLVQ.fit(X,Y)
print(confusion_matrix(YTrain, pipeGMLVQ.predict(z_train_explore), normalize='true', labels=labs))
print(confusion_matrix(YTest, pipeGMLVQ.predict(z_test_explore), normalize='true', labels=labs))

[[0.88571429 0.02857143 0.         0.02857143 0.05714286]
 [0.         0.95       0.         0.         0.05      ]
 [0.         0.07692308 0.92307692 0.         0.        ]
 [0.11764706 0.05882353 0.05882353 0.70588235 0.05882353]
 [0.08333333 0.         0.         0.         0.91666667]]
[[0.375      0.0625     0.25       0.0625     0.25      ]
 [0.         0.5        0.375      0.125      0.        ]
 [0.33333333 0.33333333 0.         0.33333333 0.        ]
 [0.2        0.         0.6        0.         0.2       ]
 [0.33333333 0.         0.33333333 0.         0.33333333]]


In [18]:

df_train_eds=df_adapted_combo[adapted_combo_cols].loc[(df_adapted_combo['Split']=='Train') & (df_adapted_combo['EDtype']!='Others')]
YTrain_eds=df_adapted_combo['EDtype'].loc[(df_adapted_combo['Split']=='Train') & (df_adapted_combo['EDtype']!='Others')]#.to_numpy()
df_test_eds=df_adapted_combo[adapted_combo_cols].loc[(df_adapted_combo['Split']=='Test') & (df_adapted_combo['EDtype']!='Others')]
YTest_eds=df_adapted_combo['EDtype'].loc[(df_adapted_combo['Split']=='Test') & (df_adapted_combo['EDtype']!='Others')]#.to_numpy()
#GridSearchClassifiers(df_train_eds, YTrain_eds, sampling_strategy)
#GridSearch_LVQ(df_train_eds, YTrain_eds, sampling_strategy)

In [23]:
nan_mean=np.nanmean(df_train_eds.to_numpy(), axis=0)
nan_std=np.nanstd(df_train_eds.to_numpy(), axis=0)
z_train_eds_nan=((df_train_eds.to_numpy()-nan_mean)/nan_std)
z_train_ed_df=pd.DataFrame(data=z_train_eds_nan, columns=adapted_combo_cols)
z_test_ed_df=pd.DataFrame(data=((df_test_eds.to_numpy()-nan_mean)/nan_std), columns=adapted_combo_cols)

kernel_mm= mf.ImputationKernel(data=z_train_ed_df,num_datasets=1,mean_match_candidates=5)
kernel_mm.mice(10)
z_train_eds=pd.DataFrame(data=kernel_mm.complete_data(), columns=adapted_combo_cols)
temp_test=kernel_mm.impute_new_data(z_test_ed_df)
z_test_eds=pd.DataFrame(data=temp_test.complete_data(), columns=adapted_combo_cols)


In [20]:
GridSearchClassifiers(z_train_eds, YTrain_eds, sampling_strategy, param_grid)

RF: 
    n_Trees  Max_Features  Lowest_CWA
5      500             7    0.637374
0      100             5    0.632828
1      300             5    0.629545
2      500             5    0.618939
KNN: 
    K Dist Metric  Lowest_CWA
3  3      cosine    0.582323
0  3   minkowski    0.518939
4  5      cosine    0.495455
5  7      cosine    0.490152
LDA: 
   Solver  Lowest_CWA
2  eigen    0.557828
0    svd    0.555556
1   lsqr    0.555556
3  eigen         NaN
Linear SVM: 
        C  Lowest_CWA
5  100.0    0.675000
4   10.0    0.669192
3    1.0    0.643434
2    0.1    0.578788
RBF SVM: 
         C  Gamma  Lowest_CWA
32  100.0  0.010    0.734343
26   10.0  0.010    0.693687
31  100.0  0.001    0.627778
20    1.0  0.010    0.535101


In [24]:
pipeRF2=Pipeline(steps=[('RF', RandomForestClassifier(criterion="gini", min_samples_leaf=5,
                        class_weight='balanced_subsample', n_estimators=500, max_features=7))])
sm2=SMOTE(sampling_strategy=sampling_strategy)
X2,Y2=sm2.fit_resample(z_train_eds, YTrain_eds)
pipeRF2.fit(X2, Y2)
print(confusion_matrix(YTrain_eds, pipeRF2.predict(z_train_eds), normalize='true', labels=np.unique(YTrain_eds)))
print(confusion_matrix(YTest_eds, pipeRF2.predict(z_test_eds), normalize='true', labels=np.unique(YTrain_eds)))

[[0.97142857 0.         0.02857143 0.        ]
 [0.05       0.85       0.05       0.05      ]
 [0.30769231 0.         0.69230769 0.        ]
 [0.         0.         0.         1.        ]]
[[0.5625     0.0625     0.1875     0.1875    ]
 [0.         0.875      0.         0.125     ]
 [0.66666667 0.33333333 0.         0.        ]
 [0.2        0.         0.4        0.4       ]]


In [22]:
param_grid_lvq2={'prot_choice': [np.array([3,2,1,1]),np.array([3,3,2,1]),np.array([3,2,2,1]),
            np.array([1,1,1,1]), np.array([1,1,2,1]), np.array([2,1, 2, 1])],
               'n_comp':[7,15, 20,29], 'scorer_name':'Lowest_CWA'}
GridSearch_LVQ(z_train_eds, YTrain_eds, sampling_strategy, param_grid_lvq2)

GMLVQ: 
      Prot per cls Solver  Num Component Activation type  Lowest_CWA
186  [2, 1, 2, 1]  lbfgs             20           swish    0.655051
106  [3, 2, 1, 1]  lbfgs             20           swish    0.646970
118  [3, 3, 2, 1]  lbfgs             15           swish    0.644697
138  [3, 2, 2, 1]  lbfgs             20           swish    0.640909


In [38]:
g_num_prots=np.array([2,1, 2, 1])
pipeGMLVQ2=Pipeline(steps=[('GMLVQ', GMLVQ(distance_type='adaptive-squared-euclidean', prototype_n_per_class=g_num_prots,
    activation_type='soft+',activation_params={"beta":3.5},relevance_n_components=20, 
                                           solver_type='lbfgs', relevance_regularization=0.01))])
pipeGMLVQ2.fit(X2, Y2)
print(confusion_matrix(YTrain_eds, pipeGMLVQ2.predict(z_train_eds), normalize='true', labels=np.unique(YTrain_eds)))
print(confusion_matrix(YTest_eds, pipeGMLVQ2.predict(z_test_eds), normalize='true', labels=np.unique(YTrain_eds)))

[[0.97142857 0.02857143 0.         0.        ]
 [0.         1.         0.         0.        ]
 [0.07692308 0.         0.84615385 0.07692308]
 [0.05882353 0.         0.         0.94117647]]
[[0.6875     0.125      0.125      0.0625    ]
 [0.125      0.875      0.         0.        ]
 [0.66666667 0.33333333 0.         0.        ]
 [0.6        0.2        0.         0.2       ]]


# Regression

In [41]:
%autoreload 2
#import importlib
#importlib.reload(
from HypOptRegress import GridSearchRegressors
param_grid_reg={'scorer': 'neg_root_mean_squared_log_error', 'RF':param_grid}
#df_train_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Train']
YTrain_reg=df_adapted_combo['EDEQ-Score'].loc[df_adapted_combo['Split']=='Train']#.to_numpy()
#df_test_adapted=df_adapted_combo[adapted_combo_cols].loc[df_adapted_combo['Split']=='Test']
YTest_reg=df_adapted_combo['EDEQ-Score'].loc[df_adapted_combo['Split']=='Test']
GridSearchRegressors(z_train_explore, YTrain_reg.to_numpy(), param_grid_reg)

KNN: 
    K Dist Metric     Score  Abs_Score
4  5      cosine -0.215046   0.215046
5  7      cosine -0.215067   0.215067
2  7   minkowski -0.225825   0.225825
0  3   minkowski -0.227075   0.227075
RF: 
     n_Trees  Max_Features      Criterion     Score  Abs_Score
12      100            25  squared_error -0.208916   0.208916
8       500            10  squared_error -0.209436   0.209436
39      100            20   friedman_mse -0.209545   0.209545
40      300            20   friedman_mse -0.209962   0.209962
Linear SVM: 
        C     Score  Abs_Score
1  0.010 -0.228766   0.228766
2  0.100 -0.254039   0.254039
0  0.001 -0.267589   0.267589
3  1.000 -0.277366   0.277366
RBF SVM: 
         C   Gamma     Score  Abs_Score
20    1.0  0.0100 -0.228277   0.228277
25   10.0  0.0010 -0.231697   0.231697
30  100.0  0.0001 -0.233006   0.233006
26   10.0  0.0100 -0.233561   0.233561


In [43]:
#df_train_eds=df_adapted_combo[adapted_combo_cols].loc[(df_adapted_combo['Split']=='Train') & (df_adapted_combo['EDtype']!='Others')]
YTrain_eds=df_adapted_combo['EDEQ-Score'].loc[(df_adapted_combo['Split']=='Train') & (df_adapted_combo['EDtype']!='Others')]#.to_numpy()
#df_test_eds=df_adapted_combo[adapted_combo_cols].loc[(df_adapted_combo['Split']=='Test') & (df_adapted_combo['EDtype']!='Others')]
YTest_eds=df_adapted_combo['EDEQ-Score'].loc[(df_adapted_combo['Split']=='Test') & (df_adapted_combo['EDtype']!='Others')]#.to_numpy()

GridSearchRegressors(z_train_eds, YTrain_eds.to_numpy(), param_grid_reg)

KNN: 
    K Dist Metric     Score  Abs_Score
4  5      cosine -0.228949   0.228949
1  5   minkowski -0.231642   0.231642
2  7   minkowski -0.236019   0.236019
5  7      cosine -0.237255   0.237255
RF: 
     n_Trees  Max_Features      Criterion     Score  Abs_Score
9       100            20  squared_error -0.213230   0.213230
39      100            20   friedman_mse -0.214002   0.214002
6       100            10  squared_error -0.214402   0.214402
56      500            20        poisson -0.214871   0.214871
Linear SVM: 
        C     Score  Abs_Score
1  0.010 -0.238631   0.238631
2  0.100 -0.265667   0.265667
0  0.001 -0.280206   0.280206
3  1.000 -0.310699   0.310699
RBF SVM: 
         C   Gamma     Score  Abs_Score
26   10.0  0.0100 -0.233543   0.233543
20    1.0  0.0100 -0.238126   0.238126
25   10.0  0.0010 -0.238259   0.238259
30  100.0  0.0001 -0.238856   0.238856
